In [ ]:
# Import Dependencies
import os
import pandas as pd


In [ ]:
# Path to file directory and variables for the two files.
file_dir = os.path.join("Data")

# imdb Titles metadata
titles_metadata_file = f'{file_dir}/title_basics_non-adult_movies.tsv'

# imdb Ratings data
ratings_data_file = f'{file_dir}/title_ratings.csv'

In [ ]:
# Import imdb Titles metadata, and imdb Ratings data

titles_metadata = pd.read_csv(titles_metadata_file, sep='\t')
#ratings_data = pd.read_csv(ratings_file)


In [ ]:
# Check DataFrames
kaggle_metadata.count()
#kaggle_metadata.head()

#ratings.count()
#ratings.head()


In [ ]:
# Keep only Non-Adult Movies
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False']

In [ ]:
# From Kaggle Data, Keep only columns: genres, id, imdb_id, title
kaggle_metadata = kaggle_metadata[['genres', 'id', 'imdb_id', 'title']]

# Add genres_list column to DataFrame
kaggle_metadata['genres_list'] = ""

# Check DataFrame
print(kaggle_metadata.shape)
kaggle_metadata.head()

In [ ]:
# Extract genres names from genres list of dicts:

replace_chars = "[']"

for index, row in kaggle_metadata.iterrows():
    genres_dicts_list = list(ast.literal_eval(row['genres']))
    genres_string = str([item['name'] for item in genres_dicts_list])
    
    for char in replace_chars:
        genres_string = genres_string.replace(char, "")
    
    row['genres_list'] = genres_string
    

print(kaggle_metadata.shape)
kaggle_metadata.head()

In [ ]:
# Locate kaggle_metadata Rows with Genres to Drop

bad_genres_list = ["Aniplex",
"BROSTA TV",
"Carousel Productions",
"Foreign",
"GoHands",
"Mardock Scramble Production Committee",
"Odyssey Media",
"Pulser Productions",
"Rogue State",
"Sentai Filmworks",
"TV Movie",
"Telescene Film Group Productions",
"The Cartel",
"Vision View Entertainment"]

kaggle_metadata[kaggle_metadata['genres_list'].str.contains('|'.join(bad_genres_list))]

In [ ]:
# Drop Rows with Genres to Drop, then Drop 'genres' Column:

kaggle_metadata = kaggle_metadata[~kaggle_metadata['genres_list'].str.contains('|'.join(bad_genres_list))]
kaggle_metadata = kaggle_metadata[['id', 'imdb_id', 'title', 'genres_list']]

# Change kaggle_metadata 'id' column to 'movieId' to match column name in
# ratings_small dataset
kaggle_metadata.rename(columns = {'id': 'movieId'}, inplace=True)

# Convert 'movieId' column to Int64
kaggle_metadata['movieId'] = kaggle_metadata['movieId'].astype(int)

print(kaggle_metadata.shape)
kaggle_metadata.head()


In [ ]:
# Check DataFrame

#kaggle_metadata['movieId'] = kaggle_metadata['movieId'].astype(int)
kaggle_metadata.dtypes

In [ ]:
# Check ratings DataFrame

print(ratings.shape)
ratings.head()
ratings.dtypes

In [ ]:
# Drop timestamp column from ratings DataFrame
# and multiply all ratings by 2 to coincide
# with IMDB 1-10 Star Ratings

ratings.drop(['timestamp'], axis=1, inplace=True)
ratings['rating'] = ratings['rating'] * 2

print(ratings.shape)
ratings.head()
ratings.dtypes

In [ ]:
# Create movies_df
movies_df = kaggle_metadata


In [ ]:
# Transform and merge the ratings DataFrame.
# Count Ratings
rating_counts = ratings.groupby(['movieId', 'rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1)

# Pivot Ratings Count Data
rating_counts = ratings.groupby(['movieId', 'rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1) \
    .pivot(index='movieId', columns='rating', values='count')

# Rename Ratings Count Columns
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

# Merge Rating Counts into movies_df
movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='movieId', right_index=True, how='left')

# Fill in Missing Ratings with Zeroes
#movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)


In [ ]:
# Check DataFrame

print(movies_with_ratings_df.shape)
movies_with_ratings_df.count()
movies_with_ratings_df.head()
#movies_with_ratings_df.tail()

In [ ]:
# Check for NULLS in movies_with_ratings_df

movies_with_ratings_df.isnull().sum()

In [ ]:
# Drop Rows with NULLS

#movies_with_ratings_df.dropna(how='any')

In [ ]:
# Look for movies which have NULLs for all Ratings

movies_with_ratings_df.loc[(movies_with_ratings_df['rating_1.0'].isnull() == True) &
                           (movies_with_ratings_df['rating_2.0'].isnull() == True)]